In [1]:
!python --version

Python 3.10.12


In [2]:
import sys

sys.path.append("/kaggle/input/monodtr-library-ver2/MonoDTR")

In [3]:
!pip install -qq -r /kaggle/input/monodtr-library-ver2/MonoDTR/requirement.txt
!pip install -qq coloredlogs
!pip install -qq ptflops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.5 MB/s eta 0:00:00


In [4]:
sys.path.append("/kaggle/input/monodtr-library-ver2/MonoDTR/scripts")
sys.path.append("/kaggle/input/pretrain-monodtr-base")
sys.path.append("/kaggle/input/checkpoint")

In [5]:
import os
import sys
import torch

# Add the directory containing the .so file to the system path
sys.path.append('/kaggle/input/monodtr-library-ver2/MonoDTR/visualDet3D/networks/lib/ops/dcn')

# Check if the .so file exists
print(os.listdir('/kaggle/input/monodtr-library-ver2/MonoDTR/visualDet3D/networks/lib/ops/dcn'))

# Print Python version
print("Python version:", sys.version)

# Set LD_LIBRARY_PATH
os.environ['LD_LIBRARY_PATH'] = '/kaggle/input/monodtr-library-ver2/MonoDTR/visualDet3D/networks/lib/ops/dcn:' + os.environ.get('LD_LIBRARY_PATH', '')

# Attempt to import the module
try:
    import deform_conv_ext
    print("Import successful!")
except ImportError as e:
    print("Import failed:", e)

# If needed, run the setup or make script
# !python setup.py build
# !python setup.py install
# or
# !bash make.sh


['deform_conv.py', 'deform_conv_ext.cpython-310-x86_64-linux-gnu.so', 'make.sh', 'src', '__init__.py', 'setup.py']
Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
Import successful!


In [6]:
import torch

if torch.cuda.is_available():
    print("CUDA is available!")
    print("Number of GPUs available:", torch.cuda.device_count())
    print("Current GPU:", torch.cuda.current_device())
    print("GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available. Check your settings.")


CUDA is available!
Number of GPUs available: 1
Current GPU: 0
GPU Name: Tesla P100-PCIE-16GB


In [7]:
# Create a large tensor and move it to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.randn(10000, 10000, device=device)  # This should allocate GPU memory
print("Tensor created on GPU.")
print("Allocated GPU Memory:", torch.cuda.memory_allocated(device))
print("Cached GPU Memory:", torch.cuda.memory_reserved(device))


Tensor created on GPU.
Allocated GPU Memory: 400556032
Cached GPU Memory: 400556032


In [8]:
import torch.nn as nn
import torch
import torch
from torch.nn import Module, Dropout
from einops import rearrange
import torch.nn.functional as F
from ptflops import get_model_complexity_info

In [9]:
# -------------------------- RMSNorm --------------------------
class RMSNorm(nn.Module):
    def __init__(self, dim, eps=1e-8):
        super().__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        # x: (B, L, D)
        norm_x = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
        return self.scale * norm_x

# -------------------------- FLIP Module --------------------------
def flip(x, dim):
    return torch.flip(x, dims=[dim])

In [10]:
# -------------------------- BiMamba2 Block (Conv Fusion) --------------------------
class BiMamba2(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.linear_x = nn.Linear(dim, dim)
        self.linear_bf = nn.Linear(dim, dim)
        self.linear_bb = nn.Linear(dim, dim)
        self.linear_z = nn.Linear(dim, dim)

        self.concat_proj_f = nn.Linear(2 * dim, dim)
        self.concat_proj_b = nn.Linear(2 * dim, dim)

        self.conv1d_f = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_b = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_zf = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_ab_z = nn.Conv1d(dim, dim, kernel_size=1)

        self.ssms_f = nn.Sequential(
            nn.Conv1d(dim, dim, 1),
            nn.GELU(),
            nn.Conv1d(dim, dim, 1)
        )
        self.ssms_b = nn.Sequential(
            nn.Conv1d(dim, dim, 1),
            nn.GELU(),
            nn.Conv1d(dim, dim, 1)
        )

        self.norm_f = RMSNorm(dim)
        self.norm_b = RMSNorm(dim)
        self.output_linear = nn.Linear(dim, dim)

    def forward(self, u):
        # u: (B, L, D)
        x = self.linear_x(u)          # (B, L, D)
        bf = self.linear_bf(u)        # (B, L, D)
        bb = self.linear_bb(u)        # (B, L, D)
        z = self.linear_z(u)          # (B, L, D)

        bf_x = torch.cat([bf, x], dim=-1)         # (B, L, 2D)
        bb_x = torch.cat([bb, x], dim=-1)         # (B, L, 2D)

        bf_x = self.concat_proj_f(bf_x)           # (B, L, D)
        bb_x = self.concat_proj_b(bb_x)           # (B, L, D)

        af = self.conv1d_f(bf_x.transpose(1, 2))  # (B, D, L)
        af = F.gelu(af)     
        af = self.ssms_f(af).transpose(1, 2)      # (B, L, D)# (B, D, L)
        # af = af + self.conv1d_zf(z.transpose(1, 2))  # af conv with z
        af = af * z
        af = self.norm_f(af)                      # (B, L, D)

        bb_x_flip = flip(bb_x, dim=1)             # (B, L, D)
        ab = self.conv1d_b(bb_x_flip.transpose(1, 2))  # (B, D, L)
        ab = F.gelu(ab)                                # (B, D, L)

        z_flip = flip(z, dim=1)                        # (B, L, D)
        ab = self.ssms_b(ab).transpose(1, 2)           # (B, L, D)
        # ab = ab + self.conv1d_ab_z(z_flip.transpose(1, 2))  # (B, D, L)
        ab = ab * z_flip   # (B, D, L)
        ab = self.norm_b(ab)                           # (B, L, D)
        ab = flip(ab, dim=1)                           # (B, L, D)

        out = self.output_linear(af + ab)         # (B, L, D)                 # (B, L, D)
        return out

In [11]:
# -------------------------- CrossMamba2 Module --------------------------
class CrossMamba2(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.linear_x = nn.Linear(dim, dim)
        self.linear_z = nn.Linear(dim, dim)
        self.linear_bf = nn.Linear(dim, dim)
        self.linear_bb = nn.Linear(dim, dim)

        self.concat_proj_f = nn.Linear(2 * dim, dim)
        self.concat_proj_b = nn.Linear(2 * dim, dim)

        self.conv1d_f = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_b = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_zf = nn.Conv1d(dim, dim, kernel_size=1)
        self.conv1d_ab_z = nn.Conv1d(dim, dim, kernel_size=1)

        self.ssms_f = nn.Sequential(
            nn.Conv1d(dim, dim, 1),
            nn.GELU(),
            nn.Conv1d(dim, dim, 1)
        )
        self.ssms_b = nn.Sequential(
            nn.Conv1d(dim, dim, 1),
            nn.GELU(),
            nn.Conv1d(dim, dim, 1)
        )

        self.norm_f = RMSNorm(dim)
        self.norm_b = RMSNorm(dim)
        self.output_linear = nn.Linear(dim, dim)

    def forward(self, u1, u2):
        # u1: context (B, L, D), u2: depth (B, L, D)
        x = self.linear_x(u1)               # (B, L, D)
        z = self.linear_z(u1)               # (B, L, D)
        bf = self.linear_bf(u2)             # (B, L, D)
        bb = self.linear_bb(u2)             # (B, L, D)

        bf_x = torch.cat([bf, x], dim=-1)   # (B, L, 2D)
        bb_x = torch.cat([bb, x], dim=-1)   # (B, L, 2D)

        bf_x = self.concat_proj_f(bf_x)     # (B, L, D)
        bb_x = self.concat_proj_b(bb_x)     # (B, L, D)

        af = self.conv1d_f(bf_x.transpose(1, 2))  # (B, D, L)
        af = F.gelu(af)
        af = self.ssms_f(af).transpose(1, 2)      # (B, L, D)
        # af = af + self.conv1d_zf(z.transpose(1, 2))  # af conv with z
        af = af * z  # af conv with z
        af = self.norm_f(af)

        bb_x_flip = flip(bb_x, dim=1)             # (B, L, D)
        ab = self.conv1d_b(bb_x_flip.transpose(1, 2))
        ab = F.gelu(ab).transpose(1, 2)           # (B, L, D)

        z_flip = flip(z, dim=1)
        ab = self.ssms_b(ab.transpose(1, 2)).transpose(1, 2)
        # ab = ab + self.conv1d_ab_z(z_flip.transpose(1, 2))  # (B, D, L)
        ab = ab * z_flip  # (B, D, L)
        ab = self.norm_b(ab)
        ab = flip(ab, dim=1)

        out = self.output_linear(af + ab)         # (B, L, D)
        return out

In [12]:
# -------------------------- FFN + Norm --------------------------
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim)
        )

    def forward(self, x):
        return self.net(x)

In [13]:
# -------------------------- Depth Aware Mamba (DTR-Compatible Input) --------------------------
class DepthAwareMamba(nn.Module):
    def __init__(self, output_channel_num):
        super().__init__()
        self.output_channel_num = output_channel_num

        self.encoder_bimamba = BiMamba2(self.output_channel_num)
        self.encoder_ffn = FeedForward(self.output_channel_num, self.output_channel_num * 2)
        self.encoder_norm1 = nn.LayerNorm(self.output_channel_num)
        self.encoder_norm2 = nn.LayerNorm(self.output_channel_num)

        self.decoder_bimamba = BiMamba2(self.output_channel_num)
        self.cross_bimamba = CrossMamba2(self.output_channel_num)
        self.decoder_ffn = FeedForward(self.output_channel_num, self.output_channel_num * 2)
        self.decoder_norm1 = nn.LayerNorm(self.output_channel_num)
        self.decoder_norm2 = nn.LayerNorm(self.output_channel_num)
        self.decoder_norm3 = nn.LayerNorm(self.output_channel_num)

    def forward(self, depth_feat, context_feat, depth_pos=None):
        depth_feat = depth_feat.contiguous()
        context_feat = context_feat.contiguous()
        if depth_pos is not None:
            depth_pos = depth_pos.contiguous()
            context_feat = context_feat + depth_pos
    
        # Encoder on context_feat
        x = self.encoder_bimamba(context_feat.contiguous())
        x = self.encoder_norm1((x + context_feat).contiguous())
        x_ffn = self.encoder_ffn(x.contiguous())
        x = self.encoder_norm2((x + x_ffn).contiguous())
    
        # Decoder on depth_feat and fused context
        d = self.decoder_bimamba(depth_feat.contiguous())
        d = self.decoder_norm1((d + depth_feat).contiguous())
    
        x = self.cross_bimamba(x.contiguous(), d.contiguous())
        x = self.decoder_norm2(x.contiguous())
        x_ffn = self.decoder_ffn(x.contiguous())
        x = self.decoder_norm3((x + x_ffn).contiguous())
        return x


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CSA(nn.Module):
    """
    Cross-Scale Attention (CSA) module.
    Projects both inputs to a common channel dimension 'c' before computing attention.
    """
    def __init__(self, c1, c2, c):
        super(CSA, self).__init__()
        self.proj_query = nn.Linear(c1, c)
        self.proj_coarser = nn.Linear(c2, c)  # Shared for key and value

    def forward(self, x1, x2):
        """
        :param x1: Finer-resolution input (B, C1, H1, W1)
        :param x2: Coarser-resolution input (B, C2, H2, W2)
        :return: Augmented finer-resolution feature (B, c, H1, W1)
        """
        B, C1, H1, W1 = x1.shape
        _, C2, H2, W2 = x2.shape

        flat1 = x1.permute(0, 2, 3, 1).reshape(B, H1 * W1, C1)
        flat2 = x2.permute(0, 2, 3, 1).reshape(B, H2 * W2, C2)

        query = self.proj_query(flat1)
        key = self.proj_coarser(flat2)
        value = self.proj_coarser(flat2)

        energy = torch.bmm(query, key.permute(0, 2, 1))
        attention = F.softmax(energy, dim=-1)

        out = torch.bmm(attention, value)

        out = out + query
        out = out.reshape(B, H1, W1, -1).permute(0, 3, 1, 2)

        return out

In [15]:
class MSR(nn.Module):
    """
    Multi-Scale Refinement (MSR) module without depth head for feature refinement only.
    """
    def __init__(self, d=256, mode='topdown'):
        super(MSR, self).__init__()
        self.mlp = nn.Sequential(
                nn.Conv2d(d, d, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm2d(d),
                nn.ReLU(inplace=True),
            )
        
        self.down_conv = nn.Sequential(
                        nn.Conv2d(d, d, kernel_size=3, stride=2, padding=1, bias=False),
                        nn.BatchNorm2d(d),
                        nn.ReLU(inplace=True),
                    )
        self.mode = mode
    def forward(self, coarser=None, finer=None):
        if self.mode == 'topdown':
            if coarser is not None:
                target_size = finer.shape[2:] if finer is not None else (coarser.shape[2] * 2, coarser.shape[3] * 2)
                up = F.interpolate(coarser, size=target_size, mode='bilinear', align_corners=False)
                if finer is not None:
                    feature = finer + up
                else:
                    feature = up
            else:
                feature = finer
        else:
            down = self.down_conv(coarser)
            feature = finer + down
            
        feature = self.mlp(feature)
        return feature 

In [16]:
from visualDet3D.networks.backbones.dla import dla102
from visualDet3D.networks.backbones.dlaup import DLAUp
from visualDet3D.networks.detectors.dfe import DepthAwareFE
from visualDet3D.networks.detectors.dpe import DepthAwarePosEnc
from visualDet3D.networks.detectors.dtr import DepthAwareTransformer
import math
import time
import csv

In [17]:
import timm

class MonoDTRCore(nn.Module):
    def __init__(self, backbone_arguments=dict()):
        super(MonoDTRCore, self).__init__()

        # Swin-T backbone with features_only=True to ensure feature pyramid output
        self.backbone = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True,features_only=True, img_size=(288, 1280))
        # self.backbone = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, features_only=True)
        self.first_level = 1  # Kept for reference, but no longer used

        # Feature pyramid channels from Swin-T
        self.channels = [96, 192, 384, 768]
        self.output_channel_num = 256

        self.csa_1_4 = CSA(96, self.output_channel_num, self.output_channel_num) # 1/4 with 1/8
        self.csa_1_8 = CSA(192, self.output_channel_num, self.output_channel_num)   # 1/8 with 1/16
        self.csa_1_16 = CSA(384, 768, self.output_channel_num)  # 1/16 with 1/32

        self.msr_top_down = MSR(d = self.output_channel_num)
        self.msr_bottom_up = MSR(d = self.output_channel_num, mode = 'bottomup')
        self.msr_depth_head = nn.Conv2d(256, 1, kernel_size=1)
        
        self.dpe = DepthAwarePosEnc(self.output_channel_num)
        self.depth_embed = nn.Embedding(100, self.output_channel_num)
        self.dtr = DepthAwareTransformer(self.output_channel_num)
        self.dfe = DepthAwareFE(self.output_channel_num)
        self.img_conv = nn.Conv2d(self.output_channel_num, self.output_channel_num, kernel_size=3, padding=1)

    def forward(self, x):
        """
        Forward pass of MonoDTRCore.
        
        Input:
            x: dict with key 'image' containing (N, 3, H, W)
        
        Output:
            feat: (N, C, H/8, W/8) - Processed feature map for compatibility with dtr
            depth: (N, D, H, W) - Depth prediction at 1/1 resolution
        """
        if 'image' not in x:
            raise ValueError("Input dictionary must contain 'image' key")
        img = x['image']  # (N, 3, H, W)
        assert img.shape[2] == 288 and img.shape[3] == 1280, f"Expected shape [N, 3, 288, 1280], got {img.shape}"

        if img.dim() != 4:
            raise ValueError(f"Expected 4D input tensor (N, C, H, W), got shape {img.shape}")
        
        if img.shape[1] != 3:
            raise ValueError(
                f"Expected input image to have 3 channels (RGB), got {img.shape[1]} channels. "
                "Please check your data pipeline to ensure the input is correctly formatted as (N, 3, H, W). "
                "If the input has 7 channels (e.g., multi-spectral), preprocess it to 3 channels or modify the backbone."
            )

        # Extract features using standard forward method
        features = self.backbone(img)  # [(N, H/4, W/4, 96), (N, H/8, W/8, 192), (N, H/16, W/16, 384), (N, H/32, W/32, 768)]

        # Permute features from (N, H, W, C) to (N, C, H, W)
        features = [f.permute(0, 3, 1, 2) for f in features]  # [(N, 96, H/4, W/4), (N, 192, H/8, W/8), (N, 384, H/16, W/16), (N, 768, H/32, W/32)]

        
        # # Refine features progressively with CSA and MSR
        f_1_16 = self.csa_1_16(features[2], features[3])  # (N, 256, H/16, W/16)
        f_1_8 = self.csa_1_8(features[1], f_1_16)     # (N, 256, H/8, W/8)
        f_1_4 = self.csa_1_4(features[0], f_1_8)     # (N, 256, H/4, W/4)

        refine_1_8_topdown = self.msr_top_down(f_1_16, f_1_8)
        x = self.msr_bottom_up(f_1_4, refine_1_8_topdown)
        refine_1_8_bottomup = self.msr_depth_head(x)
        # Proceed with downstream processing
        N, C, H, W = x.shape  # H=H/8, W=W/8 (reduced resolution)
        # print(f"X shape: {x.shape}")
        depth, depth_guide, depth_feat = self.dfe(x) # depth: (N, D, H/8, W/8), depth_guide: (N, num_classes, H/8, W/8), depth_feat: (N, 256, H/8, W/8)
        depth_feat = depth_feat.permute(0, 2, 3, 1).view(N, H * W, C)
        depth_guide = depth_guide.argmax(1)
        depth_emb = self.depth_embed(depth_guide).view(N, H * W, C)
        depth_emb = self.dpe(depth_emb, (H, W))
        img_feat = x + self.img_conv(x)
        img_feat = img_feat.permute(0, 2, 3, 1)
        # print(f"img_feat shape: {img_feat.shape}")
        img_feat = img_feat.view(N, H * W, C)
        feat = self.dtr(depth_feat, img_feat, depth_emb)
        feat = feat.permute(0, 2, 1).view(N, C, H, W)

        return feat, depth, refine_1_8_bottomup

In [18]:
from visualDet3D.networks.heads.detection_3d_head import AnchorBasedDetection3DHead
from visualDet3D.networks.heads.depth_losses import bin_depths, DepthFocalLoss
from visualDet3D.networks.utils.registry import DETECTOR_DICT

In [19]:
class MonoDTR(nn.Module):
    def __init__(self, network_cfg):
        super(MonoDTR, self).__init__()

        self.obj_types = network_cfg.obj_types

        self.build_head(network_cfg)

        self.build_core(network_cfg)

        self.network_cfg = network_cfg

    def build_core(self, network_cfg):
        self.mono_core = MonoDTRCore(network_cfg.mono_backbone)

    def build_head(self, network_cfg):
        self.bbox_head = AnchorBasedDetection3DHead(
            **(network_cfg.head)
        )
        self.depth_loss = DepthFocalLoss(96)

    def train_forward(self, left_images, annotations, P2, depth_gt=None, depth_gt_1_8 = None):
        
        features, depth, msr_output = self.mono_core(dict(image=left_images, P2=P2))
        depth_output   = depth
        
        features = features.contiguous()
        P2 = P2.contiguous()
        left_images = left_images.contiguous()
        
        try:
            cls_preds, reg_preds = self.bbox_head(
                dict(
                    features=features,
                    P2=P2,
                    image=left_images
                )
            )
        except RuntimeError as e:
            print(f"RuntimeError: {e}")
            raise
            
        anchors = self.bbox_head.get_anchor(left_images, P2)

        cls_loss, reg_loss, loss_dict = self.bbox_head.loss(cls_preds, reg_preds, anchors, annotations, P2)
        depth_gt = bin_depths(depth_gt, mode = "LID", depth_min=1, depth_max=80, num_bins=96, target=True)
        
        # if depth_gt is not None: #use in feature size is (H/4, W/4)
        #         depth_gt = F.interpolate(depth_gt.unsqueeze(1), size=depth_output.shape[2:], mode='nearest').squeeze(1)
            
        if reg_loss.mean() > 0 and not depth_gt is None and not depth_output is None:
            
            depth_gt = depth_gt.unsqueeze(1)
            depth_loss = 1.0 * self.depth_loss(depth_output, depth_gt)
            loss_dict['depth_loss'] = depth_loss
            reg_loss += depth_loss

            self.depth_output = depth_output.detach()
        else:
            loss_dict['depth_loss'] = torch.zeros_like(reg_loss)

        if depth_gt_1_8 is not None:
            # Auxiliary loss mới từ MSR (chỉ 1 scale 1/8)
            msr_loss = square_root_log_loss(msr_output, depth_gt_1_8)
            loss_dict['msr_loss'] = msr_loss
        return cls_loss, reg_loss, msr_loss, loss_dict

    def test_forward(self, left_images, P2):
        assert left_images.shape[0] == 1 # we recommmend image batch size = 1 for testing

        features, _, _ = self.mono_core(dict(image=left_images, P2=P2))
        
        cls_preds, reg_preds = self.bbox_head(
                dict(
                    features=features,
                    P2=P2,
                    image=left_images
                )
            )

        anchors = self.bbox_head.get_anchor(left_images, P2)

        scores, bboxes, cls_indexes = self.bbox_head.get_bboxes(cls_preds, reg_preds, anchors, P2, left_images)
        
        return scores, bboxes, cls_indexes

    def forward(self, inputs):

        if isinstance(inputs, list) and len(inputs) >= 3:
            return self.train_forward(*inputs)
        else:
            return self.test_forward(*inputs)

In [20]:
import csv
from typing import List, Tuple
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.utils.data
from visualDet3D.utils.utils import alpha2theta_3d, theta2alpha_3d
from visualDet3D.data.kitti.kittidata import KittiData, KittiObj, KittiCalib
from visualDet3D.data.pipeline import build_augmentator
import os
import pickle
import numpy as np
from copy import deepcopy
from visualDet3D.networks.utils import BBox3dProjector
from visualDet3D.networks.utils.registry import DATASET_DICT
import sys
from matplotlib import pyplot as plt

In [21]:
class My_KittiMonoDataset(torch.utils.data.Dataset):
    """Some Information about KittiDataset"""
    def __init__(self, cfg, split='training', data_dir = 'training'):
        super(My_KittiMonoDataset, self).__init__()
        # preprocessed_path   = cfg.path.preprocessed_path
        preprocessed_path   = "/kaggle/input/monodtr-dataset/MonoDTR/output"  #hoanbi1
        obj_types           = cfg.obj_types
        is_train = (split == 'training')
        self.data_dir = data_dir

        if split == 'training':
            imdb_file_path = os.path.join(preprocessed_path, data_dir, 'imdb.pkl')
        else:
            imdb_file_path = os.path.join(preprocessed_path, 'validation_test', 'imdb.pkl')
        self.imdb = pickle.load(open(imdb_file_path, 'rb')) # list of kittiData
        self.output_dict = {
                "calib": False,
                "image": True,
                "label": False,
                "velodyne": False
            }
        if is_train:
            self.transform = build_augmentator(cfg.data.train_augmentation)
        else:
            self.transform = build_augmentator(cfg.data.test_augmentation)
        self.projector = BBox3dProjector()
        self.is_train = is_train
        self.obj_types = obj_types
        self.use_right_image = False#= getattr(cfg.data, 'use_right_image', True)
        self.is_reproject = getattr(cfg.data, 'is_reproject', True) # if reproject 2d
        self.preprocessed_path = preprocessed_path

    def _reproject(self, P2:np.ndarray, transformed_label:List[KittiObj]) -> Tuple[List[KittiObj], np.ndarray]:
        bbox3d_state = np.zeros([len(transformed_label), 7]) #[camera_x, camera_y, z, w, h, l, alpha]
        for obj in transformed_label:
            obj.alpha = theta2alpha_3d(obj.ry, obj.x, obj.z, P2)
        bbox3d_origin = torch.tensor([[obj.x, obj.y - 0.5 * obj.h, obj.z, obj.w, obj.h, obj.l, obj.alpha] for obj in transformed_label], dtype=torch.float32)
        abs_corner, homo_corner, _ = self.projector(bbox3d_origin, bbox3d_origin.new(P2))
        for i, obj in enumerate(transformed_label):
            extended_center = np.array([obj.x, obj.y - 0.5 * obj.h, obj.z, 1])[:, np.newaxis] #[4, 1]
            extended_bottom = np.array([obj.x, obj.y, obj.z, 1])[:, np.newaxis] #[4, 1]
            image_center = (P2 @ extended_center)[:, 0] #[3]
            image_center[0:2] /= image_center[2]

            image_bottom = (P2 @ extended_bottom)[:, 0] #[3]
            image_bottom[0:2] /= image_bottom[2]
            
            bbox3d_state[i] = np.concatenate([image_center,
                                                [obj.w, obj.h, obj.l, obj.alpha]]) #[7]

        max_xy, _= homo_corner[:, :, 0:2].max(dim = 1)  # [N,2]
        min_xy, _= homo_corner[:, :, 0:2].min(dim = 1)  # [N,2]

        result = torch.cat([min_xy, max_xy], dim=-1) #[:, 4]

        bbox2d = result.cpu().numpy()

        if self.is_reproject:
            for i in range(len(transformed_label)):
                transformed_label[i].bbox_l = bbox2d[i, 0]
                transformed_label[i].bbox_t = bbox2d[i, 1]
                transformed_label[i].bbox_r = bbox2d[i, 2]
                transformed_label[i].bbox_b = bbox2d[i, 3]
        
        return transformed_label, bbox3d_state


    def __getitem__(self, index):
        kitti_data = self.imdb[index % len(self.imdb)]
        # The calib and label has been preloaded to minimize the time in each indexing
        if index >= len(self.imdb):
            kitti_data.output_dict = {
                "calib": True,
                "image": False,
                "image_3":True,
                "label": False,
                "velodyne": False
            }
            calib, _, image, _, _ = kitti_data.read_data()
            calib.P2 = calib.P3 # a workaround to use P3 for right camera images. 3D bboxes are the same(cx, cy, z, w, h, l, alpha)
        else:
            kitti_data.output_dict = self.output_dict
            _, image, _, _ = kitti_data.read_data()
            calib = kitti_data.calib
        calib.image_shape = image.shape
        label = kitti_data.label # label: list of kittiObj
        label = []
        for obj in kitti_data.label:
            if obj.type in self.obj_types:
                label.append(obj)
        transformed_image, transformed_P2, transformed_label = self.transform(image, p2=deepcopy(calib.P2), labels=deepcopy(label))
        bbox3d_state = np.zeros([len(transformed_label), 7]) #[camera_x, camera_y, z, w, h, l, alpha]
        if len(transformed_label) > 0:
            transformed_label, bbox3d_state = self._reproject(transformed_P2, transformed_label)

        ori_p2 = calib.P2

        if self.is_train:
            if (ori_p2[0, 3] * transformed_P2[0, 3]) >= 0: # not mirrored or swaped, depth should base on pointclouds projecting through P2
                depth = cv2.imread(os.path.join(self.preprocessed_path, self.data_dir, 'depth', "P2%06d.png" % index), -1)
                depth_1_8 = cv2.imread(os.path.join(self.preprocessed_path, self.data_dir, 'depth_1_8', "P2%06d.png" % index), -1)
            else: # mirrored and swap, depth should base on pointclouds projecting through P3, and also mirrored
                depth = cv2.imread(os.path.join(self.preprocessed_path, self.data_dir, 'depth', "P2%06d.png" % index), -1)
                depth = depth[:, ::-1]
                depth_1_8 = cv2.imread(os.path.join(self.preprocessed_path, self.data_dir, 'depth_1_8', "P2%06d.png" % index), -1)
                depth_1_8 = depth_1_8[:, ::-1]
        else:
            depth = None
            depth_1_8 = None

        bbox2d = np.array([[obj.bbox_l, obj.bbox_t, obj.bbox_r, obj.bbox_b] for obj in transformed_label])
        
        output_dict = {'calib': transformed_P2,
                       'image': transformed_image,
                       'label': [obj.type for obj in transformed_label], 
                       'bbox2d': bbox2d, #[N, 4] [x1, y1, x2, y2]
                       'bbox3d': bbox3d_state, 
                       'original_shape':image.shape,
                       'depth':depth,
                       'depth_1_8':depth_1_8,
                       'original_P':calib.P2.copy()}
        return output_dict

    def __len__(self):
        if self.is_train and self.use_right_image:
            return len(self.imdb) * 2
        else:
            return len(self.imdb)

    @staticmethod
    def collate_fn(batch):
        rgb_images = np.array([item["image"] for item in batch])#[batch, H, W, 3]
        rgb_images = rgb_images.transpose([0, 3, 1, 2])

        calib = [item["calib"] for item in batch]
        label = [item['label'] for item in batch]
        bbox2ds = [item['bbox2d'] for item in batch]
        bbox3ds = [item['bbox3d'] for item in batch]
        depths = [item['depth'] for item in batch]
        depths_1_8 = [item['depth_1_8'] for item in batch]
        if depths[0] is None:
            return torch.from_numpy(rgb_images).float(), torch.tensor(calib).float(), label, bbox2ds, bbox3ds
        else:
            return torch.from_numpy(rgb_images).float(), torch.tensor(calib).float(), label, bbox2ds, bbox3ds, torch.tensor(depths).float(), torch.tensor(depths_1_8).float()

In [22]:
@DATASET_DICT.register_module
class My_Val_KittiMonoDataset(torch.utils.data.Dataset):
    """Some Information about KittiDataset"""
    def __init__(self, cfg, split='training'):
        super(My_Val_KittiMonoDataset, self).__init__()
        # preprocessed_path   = cfg.path.preprocessed_path
        preprocessed_path   = "/kaggle/input/monodtr-dataset/MonoDTR/output"  #hoanbi1
        obj_types           = cfg.obj_types
        is_train = (split == 'training')

        if split == 'validation':
            imdb_file_path = os.path.join(preprocessed_path, 'validation_test', 'imdb.pkl')
        else:
            imdb_file_path = os.path.join(preprocessed_path, split, 'imdb.pkl')
        self.imdb = pickle.load(open(imdb_file_path, 'rb')) # list of kittiData
        self.output_dict = {
                "calib": False,
                "image": True,
                "label": False,
                "velodyne": False
            }
        if is_train:
            self.transform = build_augmentator(cfg.data.train_augmentation)
        else:
            self.transform = build_augmentator(cfg.data.test_augmentation)
        self.projector = BBox3dProjector()
        self.is_train = is_train
        self.obj_types = obj_types
        self.use_right_image = False#= getattr(cfg.data, 'use_right_image', True)
        self.is_reproject = getattr(cfg.data, 'is_reproject', True) # if reproject 2d
        self.preprocessed_path = preprocessed_path

    def _reproject(self, P2:np.ndarray, transformed_label:List[KittiObj]) -> Tuple[List[KittiObj], np.ndarray]:
        bbox3d_state = np.zeros([len(transformed_label), 7]) #[camera_x, camera_y, z, w, h, l, alpha]
        for obj in transformed_label:
            obj.alpha = theta2alpha_3d(obj.ry, obj.x, obj.z, P2)
        bbox3d_origin = torch.tensor([[obj.x, obj.y - 0.5 * obj.h, obj.z, obj.w, obj.h, obj.l, obj.alpha] for obj in transformed_label], dtype=torch.float32)
        abs_corner, homo_corner, _ = self.projector(bbox3d_origin, bbox3d_origin.new(P2))
        for i, obj in enumerate(transformed_label):
            extended_center = np.array([obj.x, obj.y - 0.5 * obj.h, obj.z, 1])[:, np.newaxis] #[4, 1]
            extended_bottom = np.array([obj.x, obj.y, obj.z, 1])[:, np.newaxis] #[4, 1]
            image_center = (P2 @ extended_center)[:, 0] #[3]
            image_center[0:2] /= image_center[2]

            image_bottom = (P2 @ extended_bottom)[:, 0] #[3]
            image_bottom[0:2] /= image_bottom[2]
            
            bbox3d_state[i] = np.concatenate([image_center,
                                                [obj.w, obj.h, obj.l, obj.alpha]]) #[7]

        max_xy, _= homo_corner[:, :, 0:2].max(dim = 1)  # [N,2]
        min_xy, _= homo_corner[:, :, 0:2].min(dim = 1)  # [N,2]

        result = torch.cat([min_xy, max_xy], dim=-1) #[:, 4]

        bbox2d = result.cpu().numpy()

        if self.is_reproject:
            for i in range(len(transformed_label)):
                transformed_label[i].bbox_l = bbox2d[i, 0]
                transformed_label[i].bbox_t = bbox2d[i, 1]
                transformed_label[i].bbox_r = bbox2d[i, 2]
                transformed_label[i].bbox_b = bbox2d[i, 3]
        
        return transformed_label, bbox3d_state


    def __getitem__(self, index):
        kitti_data = self.imdb[index % len(self.imdb)]
        # The calib and label has been preloaded to minimize the time in each indexing
        if index >= len(self.imdb):
            kitti_data.output_dict = {
                "calib": True,
                "image": False,
                "image_3":True,
                "label": False,
                "velodyne": False
            }
            calib, _, image, _, _ = kitti_data.read_data()
            calib.P2 = calib.P3 # a workaround to use P3 for right camera images. 3D bboxes are the same(cx, cy, z, w, h, l, alpha)
        else:
            kitti_data.output_dict = self.output_dict
            _, image, _, _ = kitti_data.read_data()
            calib = kitti_data.calib
        calib.image_shape = image.shape
        label = kitti_data.label # label: list of kittiObj
        label = []
        for obj in kitti_data.label:
            if obj.type in self.obj_types:
                label.append(obj)
        transformed_image, transformed_P2, transformed_label = self.transform(image, p2=deepcopy(calib.P2), labels=deepcopy(label))
        bbox3d_state = np.zeros([len(transformed_label), 7]) #[camera_x, camera_y, z, w, h, l, alpha]
        if len(transformed_label) > 0:
            transformed_label, bbox3d_state = self._reproject(transformed_P2, transformed_label)

        ori_p2 = calib.P2

        if self.is_train:
            if (ori_p2[0, 3] * transformed_P2[0, 3]) >= 0: # not mirrored or swaped, depth should base on pointclouds projecting through P2
                depth = cv2.imread(os.path.join(self.preprocessed_path, 'training', 'depth', "P2%06d.png" % index), -1)
            else: # mirrored and swap, depth should base on pointclouds projecting through P3, and also mirrored
                depth = cv2.imread(os.path.join(self.preprocessed_path, 'training', 'depth', "P2%06d.png" % index), -1)
                depth = depth[:, ::-1]
        else:
            depth = None

        bbox2d = np.array([[obj.bbox_l, obj.bbox_t, obj.bbox_r, obj.bbox_b] for obj in transformed_label])
        
        output_dict = {'calib': transformed_P2,
                       'image': transformed_image,
                       'label': [obj.type for obj in transformed_label], 
                       'bbox2d': bbox2d, #[N, 4] [x1, y1, x2, y2]
                       'bbox3d': bbox3d_state, 
                       'original_shape':image.shape,
                       'depth':depth,
                       'original_P':calib.P2.copy()}
        return output_dict

    def __len__(self):
        if self.is_train and self.use_right_image:
            return len(self.imdb) * 2
        else:
            return len(self.imdb)

    @staticmethod
    def collate_fn(batch):
        rgb_images = np.array([item["image"] for item in batch])#[batch, H, W, 3]
        rgb_images = rgb_images.transpose([0, 3, 1, 2])

        calib = [item["calib"] for item in batch]
        label = [item['label'] for item in batch]
        bbox2ds = [item['bbox2d'] for item in batch]
        bbox3ds = [item['bbox3d'] for item in batch]
        depths = [item['depth'] for item in batch]
        if depths[0] is None:
            return torch.from_numpy(rgb_images).float(), torch.tensor(calib).float(), label, bbox2ds, bbox3ds
        else:
            return torch.from_numpy(rgb_images).float(), torch.tensor(calib).float(), label, bbox2ds, bbox3ds, torch.tensor(depths).float()

In [23]:
import torch
import torch.nn.functional as F
def square_root_log_loss(pred, gt, lambda_bal=0.85):
    """
    Implement square root loss in log space from SideRT paper.
    pred: MSR output at 1/8 [B, 1, H/8, W/8] (clamped to positive)
    gt: Downsampled GT [B, 1, H/8, W/8]
    """
    # Clamp pred to avoid log(0) and match depth range (e.g., KITTI 0-80m)
    pred = torch.clamp(pred, min=1e-3, max=80.0)
    
    # Mask valid pixels (GT > 0)
    gt = gt.unsqueeze(1)        # [B, 1, H, W]
    mask = gt > 0
    if mask.sum() == 0:
        return torch.tensor(0.0, device=pred.device)
    
    # Flatten to 1D for sum
    log_pred = torch.log(pred[mask] + 1e-8)  # [num_valid]
    log_gt = torch.log(gt[mask] + 1e-8)      # [num_valid]
    d = log_pred - log_gt                    # d_i
    
    n = mask.sum().float()  # N_v
    
    term1 = (d ** 2).sum() / n  # (1/n) sum d_i^2
    term2 = lambda_bal / n ** 2 * (d.sum()) ** 2  # (λ / n^2) (sum d_i)^2
    
    loss = torch.sqrt(term1 - term2 + 1e-8)  # +epsilon to avoid sqrt(negative)
    return loss

In [24]:
import os
import sys
import numpy as np
from easydict import EasyDict
from tqdm import tqdm
from fire import Fire
import coloredlogs
import logging
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from _path_init import *
from visualDet3D.networks.utils.registry import DETECTOR_DICT, DATASET_DICT, PIPELINE_DICT
from visualDet3D.networks.utils.utils import BackProjection, BBox3dProjector, get_num_parameters
from visualDet3D.evaluator.kitti.evaluate import evaluate
import visualDet3D.data.kitti.dataset
from visualDet3D.utils.timer import Timer
from visualDet3D.utils.utils import LossLogger, cfg_from_file
from visualDet3D.networks.optimizers import optimizers, schedulers

In [25]:
import torch
from torch.cuda.amp import autocast  # For AMP
import torch.optim as optim
from visualDet3D.utils.utils import LossLogger
from visualDet3D.utils.utils import compound_annotation

def My_train_mono_detection(data, module: nn.Module,
                         optimizer: optim.Optimizer,
                         writer: SummaryWriter = None,
                         loss_logger: LossLogger = None,
                         global_step: int = None,
                         epoch_num: int = None,
                         cfg: EasyDict = EasyDict(),
                         iter_num: int = None,
                         val = False):
    if val == False:
        if iter_num is None:
            iter_num = 0  # Fallback, but manage externally
        steps = cfg.trainer.accumulation_steps
        is_accum_start = (iter_num % steps == 0)
        if is_accum_start and optimizer is not None:
            optimizer.zero_grad(set_to_none=True)  # Faster than zero_grad()
    
        images, P2, labels, bbox2d, bbox_3d, depth, depth_1_8 = data
        # Assume batch=1, so lists of len=1
    
        # Handle empty annotations gracefully
        max_length = np.max([len(label) for label in labels]) if labels else 0
        if max_length == 0:
            return  # Still increments iter_num externally
    
        annotation = compound_annotation(labels, max_length, bbox2d, bbox_3d, cfg.obj_types)
        annotation_tensor = images.new(annotation).cuda()  # Inherit dtype/device
    
        # Forward pass with AMP for efficiency
        # with autocast(device_type='cuda', enabled=cfg.get('use_amp', True)):
        cls_loss, reg_loss, msr_loss, loss_dict = module([
            images.cuda().float().contiguous(),
            annotation_tensor,
            P2.cuda(),
            depth.cuda().contiguous(),
            depth_1_8.cuda().contiguous()
        ])
        cls_loss = cls_loss.mean()
        reg_loss = reg_loss.mean()
        msr_loss = msr_loss.mean()
        total_loss = cls_loss + reg_loss + msr_loss
        # loss_dict['total_loss'] = total_loss.item()  # Add for logging
    
        # Log unscaled micro-batch losses
        if loss_logger is not None:
            loss_logger.update(loss_dict)
        del loss_dict
    
        if optimizer is not None:
            # Check for invalid losses
            if torch.isnan(total_loss) or torch.isinf(total_loss):
                print(f"Warning: Invalid loss at iter {iter_num}: {total_loss.item()}")
                # Optionally: optimizer.zero_grad(); return
                total_loss = torch.tensor(0.0, device=total_loss.device, requires_grad=True)
    
            scaled_loss = total_loss / steps
            scaled_loss.backward()
    
        # Update every accumulation_steps
        if (iter_num + 1) % steps == 0:
            # Gradient clipping
            if hasattr(cfg.optimizer, 'clipped_gradient_norm'):
                torch.nn.utils.clip_grad_norm_(module.parameters(), cfg.optimizer.clipped_gradient_norm)
            else:
                print("Warning: No clipped_gradient_norm in cfg; skipping clipping.")
    
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)  # Reset after step for next cycle     
    else:
        images, P2, labels, bbox2d, bbox_3d, depth, depth_1_8 = data
        
        max_length = np.max([len(label) for label in labels]) if labels else 0
        if max_length == 0:
            return  # Still increments iter_num externally
    
        annotation = compound_annotation(labels, max_length, bbox2d, bbox_3d, cfg.obj_types)
        annotation_tensor = images.new(annotation).cuda()  # Inherit dtype/device
    
        cls_loss, reg_loss, msr_loss, loss_dict = module([
            images.cuda().float().contiguous(),
            annotation_tensor,
            P2.cuda(),
            depth.cuda().contiguous(),
            depth_1_8.cuda().contiguous()
        ])
        
        cls_loss = cls_loss.mean()
        reg_loss = reg_loss.mean()
        msr_loss = msr_loss.mean()
        total_loss = cls_loss + reg_loss + msr_loss
        # loss_dict['total_loss'] = total_loss.item()  # Add for logging
    
        # Log unscaled micro-batch losses
        if loss_logger is not None:
            loss_logger.update(loss_dict, data_split='val')
        del loss_dict 

In [26]:
import torch
import numpy as np
import cv2
import sys
import os
import tempfile
import shutil
import importlib
from easydict import EasyDict

def convertAlpha2Rot(alpha, cx, P2):
    cx_p2 = P2[..., 0, 2]
    fx_p2 = P2[..., 0, 0]
    ry3d = alpha + np.arctan2(cx - cx_p2, fx_p2)
    ry3d[np.where(ry3d > np.pi)] -= 2 * np.pi
    ry3d[np.where(ry3d <= -np.pi)] += 2 * np.pi
    return ry3d


def convertRot2Alpha(ry3d, cx, P2):
    cx_p2 = P2[..., 0, 2]
    fx_p2 = P2[..., 0, 0]
    alpha = ry3d - np.arctan2(cx - cx_p2, fx_p2)
    alpha[alpha > np.pi] -= 2 * np.pi
    alpha[alpha <= -np.pi] += 2 * np.pi
    return alpha

def alpha2theta_3d(alpha, x, z, P2):
    """ Convert alpha to theta with 3D position
    Args:
        alpha [torch.Tensor/ float or np.ndarray]: size: [...]
        x     []: size: [...]
        z     []: size: [...]
        P2    [torch.Tensor/ np.ndarray]: size: [3, 4]
    Returns:
        theta []: size: [...]
    """
    offset = P2[0, 3] / P2[0, 0]
    if isinstance(alpha, torch.Tensor):
        theta = alpha + torch.atan2(x + offset, z)
    else:
        theta = alpha + np.arctan2(x + offset, z)
    return theta

def theta2alpha_3d(theta, x, z, P2):
    """ Convert theta to alpha with 3D position
    Args:
        theta [torch.Tensor/ float or np.ndarray]: size: [...]
        x     []: size: [...]
        z     []: size: [...]
        P2    [torch.Tensor/ np.ndarray]: size: [3, 4]
    Returns:
        alpha []: size: [...]
    """
    offset = P2[0, 3] / P2[0, 0]
    if isinstance(theta, torch.Tensor):
        alpha = theta - torch.atan2(x + offset, z)
    else:
        alpha = theta - np.arctan2(x + offset, z)
    return alpha

def draw_3D_box(img, corners, color = (255, 255, 0)):
    """
        draw 3D box in image with OpenCV,
        the order of the corners should be the same with BBox3dProjector
    """
    points = np.array(corners[0:2], dtype=np.int32) #[2, 8]
    points = [tuple(points[:,i]) for i in range(8)]
    for i in range(1, 5):
        cv2.line(img, points[i], points[(i%4+1)], color, 2)
        cv2.line(img, points[(i + 4)%8], points[((i)%4 + 5)%8], color, 2)
    cv2.line(img, points[2], points[7], color)
    cv2.line(img, points[3], points[6], color)
    cv2.line(img, points[4], points[5],color)
    cv2.line(img, points[0], points[1], color)
    return img

def compound_annotation(labels, max_length, bbox2d, bbox_3d, obj_types):
    """ Compound numpy-like annotation formats. Borrow from Retina-Net
    
    Args:
        labels: List[List[str]]
        max_length: int, max_num_objects, can be dynamic for each iterations
        bbox_2d: List[np.ndArray], [left, top, right, bottom].
        bbox_3d: List[np.ndArray], [cam_x, cam_y, z, w, h, l, alpha].
        obj_types: List[str]
    Return:
        np.ndArray, [batch_size, max_length, 12]
            [x1, y1, x2, y2, cls_index, cx, cy, z, w, h, l, alpha]
            cls_index = -1 if empty
    """
    annotations = np.ones([len(labels), max_length, bbox_3d[0].shape[-1] + 5]) * -1
    for i in range(len(labels)):
        label = labels[i]
        for j in range(len(label)):
            annotations[i, j] = np.concatenate([
                bbox2d[i][j], [obj_types.index(label[j])], bbox_3d[i][j]
            ])
    return annotations

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def cfg_from_file(cfg_filename:str)->EasyDict:
    assert cfg_filename.endswith('.py')

    with tempfile.TemporaryDirectory() as temp_config_dir:
        temp_config_file = tempfile.NamedTemporaryFile(dir=temp_config_dir, suffix='.py')
        temp_config_name = os.path.basename(temp_config_file.name)
        shutil.copyfile(cfg_filename, os.path.join(temp_config_dir, temp_config_name))
        temp_module_name = os.path.splitext(temp_config_name)[0]
        sys.path.insert(0, temp_config_dir)
        cfg = getattr(importlib.import_module(temp_module_name), 'cfg')
        assert isinstance(cfg, EasyDict)
        sys.path.pop()
        del sys.modules[temp_module_name]
        temp_config_file.close()

    return cfg

In [27]:
class My_LossLogger():
    def __init__(self, recorder):
        self.recorder = recorder
        self.reset()

    def reset(self):
        self.loss_stats = {
            'train': {},
            'val': {}
        }  # each split will have its own dict of losses

    def update(self, loss_dict, data_split='train'):
        if data_split not in self.loss_stats:
            raise ValueError(f"Unsupported data_split: {data_split}")
        
        for key in loss_dict:
            # Prefix the key with the split to avoid conflicts, but since we're nesting, no need
            # But to keep keys unique per split, we use nested dicts
            if key not in self.loss_stats[data_split]:
                self.loss_stats[data_split][key] = AverageMeter()
            self.loss_stats[data_split][key].update(loss_dict[key].mean().item())

    def log(self, step):
        for split in self.loss_stats:
            for key in self.loss_stats[split]:
                name = key + '/' + split
                self.recorder.add_scalar(name, self.loss_stats[split][key].avg, step)

In [28]:
from collections import OrderedDict  # Add this import
import matplotlib.pyplot as plt
import cv2
from skimage.measure import block_reduce
def main(config="/kaggle/input/monodtr-library-ver2/MonoDTR/config/config.py", experiment_name="default", world_size=1, local_rank=-1):
    """Main function for the training script.

    KeywordArgs:
        config (str): Path to config file.
        experiment_name (str): Custom name for the experitment, only used in tensorboard.
        world_size (int): Number of total subprocesses in distributed training. 
        local_rank: Rank of the process. Should not be manually assigned. 0-N for ranks in distributed training (only process 0 will print info and perform testing). -1 for single training. 
    """

    ## Get config
    cfg = cfg_from_file(config)

    ## Collect distributed(or not) information
    cfg.dist = EasyDict()
    cfg.dist.world_size = world_size
    cfg.dist.local_rank = local_rank
    is_distributed = local_rank >= 0 # local_rank < 0 -> single training
    is_logging     = local_rank <= 0 # only log and test with main process
    is_evaluating  = local_rank <= 0

    ## Setup writer if local_rank > 0
    recorder_dir = os.path.join(cfg.path.log_path, experiment_name + f"config={config}")
    if is_logging: # writer exists only if not distributed and local rank is smaller
        ## Clean up the dir if it exists before
        if os.path.isdir(recorder_dir):
            os.system("rm -r {}".format(recorder_dir))
            print("clean up the recorder directory of {}".format(recorder_dir))
        writer = SummaryWriter(recorder_dir)

        ## Record config object using pprint
        import pprint

        formatted_cfg = pprint.pformat(cfg)
        writer.add_text("config.py", formatted_cfg.replace(' ', '&nbsp;').replace('\n', '  \n')) # add space for markdown style in tensorboard text
    else:
        writer = None

    ## Set up GPU and distribution process
    if is_distributed:
        cfg.trainer.gpu = local_rank # local_rank will overwrite the GPU in configure file
    gpu = min(cfg.trainer.gpu, torch.cuda.device_count() - 1)
    torch.backends.cudnn.benchmark = getattr(cfg.trainer, 'cudnn', False)
    torch.cuda.set_device(gpu)
    if is_distributed:
        torch.distributed.init_process_group(backend='nccl', init_method='env://')
    print(local_rank)
 
    ## define datasets and dataloader.
    dataset_train = My_KittiMonoDataset(cfg, data_dir = 'training')
    dataset_val_train = My_KittiMonoDataset(cfg, data_dir = 'validation')
    dataset_val = My_Val_KittiMonoDataset(cfg, "validation")
    # dataset_train = DATASET_DICT[cfg.data.train_dataset](cfg)
    # dataset_val = DATASET_DICT[cfg.data.val_dataset](cfg, "validation")

    dataloader_train = DataLoader(dataset_train, num_workers=cfg.data.num_workers,
                                  batch_size=cfg.data.batch_size, collate_fn=dataset_train.collate_fn, shuffle=local_rank<0, drop_last=True,
                                  sampler=torch.utils.data.DistributedSampler(dataset_train, num_replicas=world_size, rank=local_rank, shuffle=True) if local_rank >= 0 else None)
    dataloader_validation = DataLoader(dataset_val_train, num_workers=cfg.data.num_workers,
                                  batch_size=cfg.data.batch_size, collate_fn=dataset_train.collate_fn, shuffle=local_rank<0, drop_last=True,
                                  sampler=torch.utils.data.DistributedSampler(dataset_train, num_replicas=world_size, rank=local_rank, shuffle=True) if local_rank >= 0 else None)

    detector = MonoDTR(cfg.detector)         
            
    ## Convert to cuda
    if is_distributed:
        detector = torch.nn.SyncBatchNorm.convert_sync_batchnorm(detector)
        detector = torch.nn.parallel.DistributedDataParallel(detector.cuda(), device_ids=[gpu], output_device=gpu)
    else:
        detector = detector.cuda()
    
    ## define optimizer and weight decay
    optimizer = optimizers.build_optimizer(cfg.optimizer, detector)

    ## define scheduler
    scheduler_config = getattr(cfg, 'scheduler', None)
    scheduler = schedulers.build_scheduler(scheduler_config, optimizer)
    is_iter_based = getattr(scheduler_config, "is_iter_based", False)

    ## define loss logger
    training_loss_logger =  My_LossLogger(writer) if is_logging else None

    ## training pipeline
    if 'training_func' in cfg.trainer:
        # training_dection = PIPELINE_DICT[cfg.trainer.training_func] #hoanbi1
        training_dection = My_train_mono_detection
    else:
        raise KeyError

    ## Get evaluation pipeline
    if 'evaluate_func' in cfg.trainer:
        evaluate_detection = PIPELINE_DICT[cfg.trainer.evaluate_func]
        print("Found evaluate function {}".format(cfg.trainer.evaluate_func))
    else:
        evaluate_detection = None
        print("Evaluate function not found")


    ## timer is used to estimate eta
    timer = Timer()

    print('Num training images: {}'.format(len(dataset_train)))

    resume = True
    epoch = 0
    freeze = False
    if resume == True:
        ckpt_path = os.path.join("/kaggle/input/monodtr-resume-checkpoint", "checkpoint_resume.pt")
        dict_checkpoint = torch.load(ckpt_path, map_location="cpu")
        detector.load_state_dict(dict_checkpoint["state_dict_backbone"])
        optimizer.load_state_dict(dict_checkpoint["state_optimizer"])
        scheduler.load_state_dict(dict_checkpoint["state_lr_scheduler"])
        epoch = dict_checkpoint["epoch"]
        print(f"Resuming training at epoch {epoch}")
        del dict_checkpoint
        
        if freeze == True:
            # this is for load pretrain checkpoint
            state_dict_pretrain = torch.load("/kaggle/input/pretrain-monodtr-base/MonoDTR.pth", map_location='cpu')
            
            new_state_dict = OrderedDict()
            loaded_keys = []  # Track which parameters are loaded
            for k, v in state_dict_pretrain.items():
                if k in detector.state_dict() and v.shape == detector.state_dict()[k].shape:
                    new_state_dict[k] = v
                    loaded_keys.append(k)  # Record loaded keys
                    print(f"Parameter {k} is loaded from pretrain")
        
            detector.load_state_dict(new_state_dict, strict=False)
            
            # for name, param in detector.named_parameters():
            #     # print(name)
            #     if name in loaded_keys:
            #         param.requires_grad = False  # Freeze loaded weights
            #     else:
            #         param.requires_grad = True
            for name, param in detector.named_parameters():
                if "mono_core.msr_top_down" in name or "mono_core.msr_bottom_up" in name:
                    param.requires_grad = True  # Freeze loaded weights
                else:
                    param.requires_grad = False
    else:
        print("Start training at epoch 0")
        if freeze == True:
            state_dict_checkpoint = torch.load("/kaggle/input/checkpoint-visualize/MonoDTR_39.pth", map_location='cpu')
            detector.load_state_dict(state_dict_checkpoint, strict = False) 
            # this is for load pretrain checkpoint
            state_dict_pretrain = torch.load("/kaggle/input/pretrain-monodtr-base/MonoDTR.pth", map_location='cpu')
            
            new_state_dict = OrderedDict()
            loaded_keys = []  # Track which parameters are loaded
            for k, v in state_dict_pretrain.items():
                if k in detector.state_dict() and v.shape == detector.state_dict()[k].shape:
                    new_state_dict[k] = v
                    loaded_keys.append(k)  # Record loaded keys
                    print(f"Parameter {k} is loaded from pretrain")
        
            detector.load_state_dict(new_state_dict, strict=False)
            
            # for name, param in detector.named_parameters():
            #     # print(name)
            #     if name in loaded_keys:
            #         param.requires_grad = False  # Freeze loaded weights
            #     else:
            #         param.requires_grad = True
            
            for name, param in detector.named_parameters():
                if "mono_core.msr_top_down" in name or "mono_core.msr_bottom_up" in name:
                    param.requires_grad = True  # Freeze loaded weights
                else:
                    param.requires_grad = False
        else:
            state_dict_pretrain = torch.load("/kaggle/input/checkpoint/MonoDTR_229.pth", map_location='cpu')
            detector.load_state_dict(state_dict_pretrain, strict = False)

    for name, param in detector.named_parameters():
        if param.requires_grad == True:
            print(f"Parameter {name} is trainable")
            
    ## Record basic information of the model
    if is_logging:
        string1 = detector.__str__().replace(' ', '&nbsp;').replace('\n', '  \n')
        writer.add_text("model structure", string1) # add space for markdown style in tensorboard text
        num_parameters = get_num_parameters(detector)
        print(f'number of trained parameters of the model: {num_parameters}')
        
    detector.train()
    global_step = 0
    
    l_rate = optimizer.param_groups[0]['lr']
    print(f"Starting LR = {l_rate:.6f}")
    
    # evaluate_detection(cfg, detector.module if is_distributed else detector, dataset_val, writer, epoch)
    
    # for epoch_num in range(cfg.trainer.max_epochs):
    for epoch_num in range(epoch, (5+epoch)):
        ## Start training for one epoch
        torch.cuda.empty_cache()
        detector.train()
        if training_loss_logger:
            training_loss_logger.reset()
                    
            for iter_num, data in enumerate(dataloader_train):
                training_dection(data, detector, optimizer, writer, training_loss_logger, global_step, epoch_num, cfg, iter_num)
                global_step += 1
                if (iter_num + 1) % cfg.trainer.accumulation_steps == 0:
                    if is_iter_based:
                        scheduler.step()
        
                    if is_logging and global_step % cfg.trainer.disp_iter == 0:
                        ## Log loss, print out and write to tensorboard in main process
                        if 'train' not in training_loss_logger.loss_stats or 'total_loss' not in training_loss_logger.loss_stats['train']:
                            print(f"\nIn epoch {epoch_num}, iteration:{iter_num}, global_step:{global_step}, total_loss not found in logger for train.")
                        else:
                            log_str = 'Epoch: {} | Iteration: {} | Running loss: {:1.5f} | eta:{}'.format(
                                epoch_num, iter_num, training_loss_logger.loss_stats['train']['total_loss'].avg,
                                timer.compute_eta(global_step, len(dataloader_train) * cfg.trainer.max_epochs))
                            
                            print(log_str, end='\r')
                            writer.add_text("training_log/train", log_str, global_step)
                            training_loss_logger.log(global_step)
                            
            # Handle partial accumulation at end of epoch
            if (iter_num + 1) % cfg.trainer.accumulation_steps != 0:
                # Gradient clipping if applicable
                if hasattr(cfg.optimizer, 'clipped_gradient_norm'):
                    torch.nn.utils.clip_grad_norm_(detector.parameters(), cfg.optimizer.clipped_gradient_norm)
                else:
                    print("Warning: No clipped_gradient_norm in cfg; skipping clipping.")
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                
            if not is_iter_based:
                scheduler.step()

            #Compute loss for validation dataset 
            for iter_num, data in enumerate(dataloader_validation):
                training_dection(data, detector, optimizer, writer, training_loss_logger, global_step, epoch_num, cfg, iter_num, val=True)
                
            if is_logging:
                if 'val' in training_loss_logger.loss_stats and 'total_loss' in training_loss_logger.loss_stats['val']:
                    log_str = 'Epoch: {} |  Running valition loss: {:1.5f}'.format(
                                epoch_num, training_loss_logger.loss_stats['val']['total_loss'].avg)
                    print(log_str, end='\r')
                    writer.add_text("training_log/validation", log_str, epoch_num)
                    training_loss_logger.log(epoch_num)
            
            l_rate = optimizer.param_groups[0]['lr']
            print(f"Epoch {epoch_num+1} — LR = {l_rate:.6f}")
            
            ## save model in main process if needed
            if is_logging:
                torch.save(detector.module.state_dict() if is_distributed else detector.state_dict(), os.path.join(
                    cfg.path.checkpoint_path, '{}_latest.pth'.format(
                        cfg.detector.name)
                    )
                )
            if is_logging and (epoch_num + 1) % cfg.trainer.save_iter == 0:
                torch.save(detector.module.state_dict() if is_distributed else detector.state_dict(), os.path.join(
                    cfg.path.checkpoint_path, '{}_{}.pth'.format(
                        cfg.detector.name,epoch_num)
                    )
                )
            checkpoint = {
                    "epoch": epoch_num + 1,
                    "global_step": global_step,
                    "state_dict_backbone": detector.state_dict(),
                    "state_optimizer": optimizer.state_dict(),
                    "state_lr_scheduler": scheduler.state_dict()
                }
            torch.save(checkpoint, os.path.join(cfg.path.checkpoint_path, f"checkpoint_resume.pt"))
            print(f"Save checkpoint at epoch {epoch_num+1} successfully!")
            ## test model in main process if needed
            if is_evaluating and evaluate_detection is not None and cfg.trainer.test_iter > 0 and (epoch_num + 1) % cfg.trainer.test_iter == 0:
                print("\n/**** start testing after training epoch {} ******/".format(epoch_num))
                evaluate_detection(cfg, detector.module if is_distributed else detector, dataset_val, writer, epoch_num)
                print("/**** finish testing after training epoch {} ******/".format(epoch_num))
    
            if is_distributed:
                torch.distributed.barrier() # wait untill all finish a epoch
    
            if is_logging:
                writer.flush()

In [29]:
%%writefile my_config.py

from easydict import EasyDict as edict
import os 
import numpy as np

cfg = edict()
cfg.obj_types = ['Car']
#cfg.obj_types = ['Car', 'Pedestrian', 'Cyclist']

## trainer
trainer = edict(
    gpu = 0,
    max_epochs = 240, 
    disp_iter = 100,
    save_iter = 5,
    test_iter = 10,
    training_func = "train_mono_detection",
    test_func = "test_mono_detection",
    evaluate_func = "evaluate_kitti_obj",
    accumulation_steps = 1,  # hoanbi1
)

cfg.trainer = trainer

## path
path = edict()
path.data_path = "/kaggle/input/kitti-3d-object-detection-dataset/training" # used in visualDet3D/data/.../dataset
path.test_path = "/kaggle/input/kitti-3d-object-detection-dataset/testing" # used in visualDet3D/data/.../dataset
path.visualDet3D_path = "/kaggle/input/monodtr-library-ver2/MonoDTR/visualDet3D" # The path should point to the inner subfolder
path.project_path = "/kaggle/working/" # or other path for pickle files, checkpoints, tensorboard logging and output files.
if not os.path.isdir(path.project_path):
    os.mkdir(path.project_path)
path.project_path = os.path.join(path.project_path, 'MonoDTR')
if not os.path.isdir(path.project_path):
    os.mkdir(path.project_path)

path.log_path = os.path.join(path.project_path, "log")
if not os.path.isdir(path.log_path):
    os.mkdir(path.log_path)

path.checkpoint_path = os.path.join(path.project_path, "checkpoint")
if not os.path.isdir(path.checkpoint_path):
    os.mkdir(path.checkpoint_path)

path.preprocessed_path = os.path.join(path.project_path, "output")
if not os.path.isdir(path.preprocessed_path):
    os.mkdir(path.preprocessed_path)

path.train_imdb_path = os.path.join(path.preprocessed_path, "training")
if not os.path.isdir(path.train_imdb_path):
    os.mkdir(path.train_imdb_path)

path.val_imdb_path = os.path.join(path.preprocessed_path, "validation")
if not os.path.isdir(path.val_imdb_path):
    os.mkdir(path.val_imdb_path)

path.test_imdb_path = os.path.join(path.preprocessed_path, "test")
if not os.path.isdir(path.test_imdb_path):
    os.mkdir(path.test_imdb_path)

path.test_result_path = os.path.join(path.test_imdb_path, "data")
if not os.path.isdir(path.test_result_path):
    os.mkdir(path.test_result_path)

cfg.path = path

## optimizer
optimizer = edict(
    type_name = 'adam',
    keywords = edict(
        lr        = 1e-4,
        weight_decay = 0,
    ),
    clipped_gradient_norm = 0.1
)
cfg.optimizer = optimizer
## scheduler
scheduler = edict(
    type_name = 'CosineAnnealingLR',
    keywords = edict(
        T_max     = cfg.trainer.max_epochs,
        eta_min   = 5e-6,
    )
)
cfg.scheduler = scheduler

## data
data = edict(
    batch_size = 4,
    num_workers = 8,
    rgb_shape = (288, 1280, 3),
    train_dataset = "My_KittiMonoDataset",
    val_dataset   = "My_KittiMonoDataset",
    test_dataset  = "KittiMonoTestDataset",
    train_split_file = os.path.join(cfg.path.visualDet3D_path, 'data', 'kitti', 'chen_split', 'train.txt'),
    val_split_file   = os.path.join(cfg.path.visualDet3D_path, 'data', 'kitti', 'chen_split', 'val.txt'),
)

data.augmentation = edict(
    rgb_mean = np.array([0.485, 0.456, 0.406]),
    rgb_std  = np.array([0.229, 0.224, 0.225]),
    cropSize = (data.rgb_shape[0], data.rgb_shape[1]),
    crop_top = 100,
)
data.train_augmentation = [
    edict(type_name='ConvertToFloat'),
    edict(type_name='PhotometricDistort', keywords=edict(distort_prob=1.0, contrast_lower=0.5, contrast_upper=1.5, saturation_lower=0.5, saturation_upper=1.5, hue_delta=18.0, brightness_delta=32)),
    edict(type_name='CropTop', keywords=edict(crop_top_index=data.augmentation.crop_top)),
    edict(type_name='Resize', keywords=edict(size=data.augmentation.cropSize)),
    edict(type_name='RandomMirror', keywords=edict(mirror_prob=0.5)),
    edict(type_name='Normalize', keywords=edict(mean=data.augmentation.rgb_mean, stds=data.augmentation.rgb_std))
]
data.test_augmentation = [
    edict(type_name='ConvertToFloat'),
    edict(type_name='CropTop', keywords=edict(crop_top_index=data.augmentation.crop_top)),
    edict(type_name='Resize', keywords=edict(size=data.augmentation.cropSize)),
    edict(type_name='Normalize', keywords=edict(mean=data.augmentation.rgb_mean, stds=data.augmentation.rgb_std))
]
cfg.data = data

## networks
detector = edict()
detector.obj_types = cfg.obj_types
detector.name = 'MonoDTR'
detector.mono_backbone=edict(
    msr_loss_weight = 1.0,  # Weight cho loss mới
    lambda_bal = 0.85,       # Từ paper
)
head_loss = edict(
    fg_iou_threshold = 0.5,
    bg_iou_threshold = 0.4,
    L1_regression_alpha = 5 ** 2,
    focal_loss_gamma = 2.0,
    balance_weight   = [20.0],
    #balance_weight   = [20.0, 40, 40],
    regression_weight = [1, 1, 1, 1, 1, 1, 12, 1, 1, 0.5, 0.5, 0.5, 1], #[x, y, w, h, cx, cy, z, sin2a, cos2a, w, h, l]
)
head_test = edict(
    score_thr=0.75,
    cls_agnostic = False,
    nms_iou_thr=0.4,
    post_optimization=False
)

anchors = edict(
        {
            'obj_types': cfg.obj_types,
            'pyramid_levels':[3],
            'strides': [2 ** 3],
            'sizes' : [24],
            'ratios': np.array([0.5, 1, 2.0]),
            'scales': np.array([2 ** (i / 4.0) for i in range(16)]),
        }
    )

head_layer = edict(
    num_features_in=256,
    num_cls_output=len(cfg.obj_types)+1,
    num_reg_output=12,
    cls_feature_size=256,
    reg_feature_size=256,
)
detector.head = edict(
    num_regression_loss_terms=13,
    preprocessed_path=path.preprocessed_path,
    num_classes     = len(cfg.obj_types),
    anchors_cfg     = anchors,
    layer_cfg       = head_layer,
    loss_cfg        = head_loss,
    test_cfg        = head_test
)
detector.anchors = anchors
detector.loss = head_loss
cfg.detector = detector


Writing my_config.py


In [30]:
def run_training():
    # config_path = "/kaggle/input/monodtr-library-ver2/MonoDTR/config/config.py"  # Path to your config file
    config_path = "/kaggle/working/my_config.py"  # Path to your config file
    experiment_name = "EXP_NAME"  # Use the defined experiment name
    world_size = 1  # For single GPU training
    local_rank = 0  # Local rank set to 0 as per your command
    %env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
    main(config=config_path, experiment_name=experiment_name, world_size=world_size)

In [31]:
run_training()

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
-1


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

Found evaluate function evaluate_kitti_obj
Num training images: 3712


<ipython-input-28-40511a0d3584>:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dict_checkpoint = torch.load(ckpt_path, map_location="cpu")


...

<ipython-input-21-4e730322d0aa>:144: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.from_numpy(rgb_images).float(), torch.tensor(calib).float(), label, bbox2ds, bbox3ds, torch.tensor(depths).float(), torch.tensor(depths_1_8).float()
<ipython-input-21-4e730322d0aa>:144: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.from_numpy(rgb_images).float(), torch.tensor(calib).float(), label, bbox2ds, bbox3ds, torch.tensor(depths).float(), torch.tensor(depths_1_8).float()
<ipython-input-21-4e730322d0aa>:144: UserWarning: Creating a tensor from a list of

Epoch 144 — LR = 0.000038
Save checkpoint at epoch 144 successfully!


<ipython-input-21-4e730322d0aa>:144: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.from_numpy(rgb_images).float(), torch.tensor(calib).float(), label, bbox2ds, bbox3ds, torch.tensor(depths).float(), torch.tensor(depths_1_8).float()
<ipython-input-21-4e730322d0aa>:144: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.from_numpy(rgb_images).float(), torch.tensor(calib).float(), label, bbox2ds, bbox3ds, torch.tensor(depths).float(), torch.tensor(depths_1_8).float()
<ipython-input-21-4e730322d0aa>:144: UserWarning: Creating a tensor from a list of

<ipython-input-21-4e730322d0aa>:144: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.from_numpy(rgb_images).float(), torch.tensor(calib).float(), label, bbox2ds, bbox3ds, torch.tensor(depths).float(), torch.tensor(depths_1_8).float()
<ipython-input-21-4e730322d0aa>:144: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.from_numpy(rgb_images).float(), torch.tensor(calib).float(), label, bbox2ds, bbox3ds, torch.tensor(depths).float(), torch.tensor(depths_1_8).float()
<ipython-input-21-4e730322d0aa>:144: UserWarning: Creating a tensor from a list of

Epoch 145 — LR = 0.000037
Save checkpoint at epoch 145 successfully!
